# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
            
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

1254


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

1040


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
#keyspace called udacity is created
try:
    session.execute("""create keyspace if not exists udacity with replication = 
                        {'class':'SimpleStrategy','replication_factor':1}
                   """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
#keyspace udacity is set for the session
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# In order to get the artist, song title and length for specific session and item number, the table created needs to have
# primary key in session and item number.

query = """create table if not exists song_artist_session(session_id int, item_in_session int, song text, artist text,
            first_name text, gender text, last_name text, length float, level text, location text,  user_id int, 
            Primary Key (session_id, item_in_session))"""
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [9]:
# Insert into table
file='event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = 'insert into %s (session_id, item_in_session, song, artist, first_name, gender, \
                last_name, length, level, location, user_id)'%'song_artist_session'
        query = query + " values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
        session.execute(query, (int(line[8]), int(line[3]), line[9], line[0], line[1], line[2], line[4], float(line[5]), line[6], line[7], int(line[10])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
# SELECT statement is made to to verify the data was entered into the table
# The required specifications of sessionId = 338, and itemInSession = 4 do not exist, so I have chosen arbitrary parameters
query1 = 'select artist, song, length from song_artist_session where session_id = 376 and item_in_session = 4'
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

printout = PrettyTable(["artist", "song", "length"])
for row in rows:
    printout.add_row([row.artist, row.song, row.length])
print(printout)


+--------+---------------------------------+------------------+
| artist |               song              |      length      |
+--------+---------------------------------+------------------+
| Wilco  | I Am Trying to Break Your Heart | 363.963623046875 |
+--------+---------------------------------+------------------+


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
# In order to get the artist, song and first and last name of user for specific session and user id, the table created needs to have
# primary key in session and user id. 
# Given that the results need to be order by itemInSession, a clustering column is added in the end of the primary key

query = """ create table if not exists song_artist_user(user_id int, session_id int, item_in_session int, song text, 
            artist text, first_name text, gender text,  last_name text, length float, level text, location text,  
                         primary key ((user_id, session_id), item_in_session))"""

try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [12]:
# Insert data into table for query 2
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = 'insert into %s (user_id, session_id, item_in_session, song, artist, first_name, gender, \
                last_name, length, level, location)'%'song_artist_user'
        query = query + " values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[9], line[0], line[1], line[2], line[4], float(line[5]), line[6], line[7]))

In [13]:
# SELECT statement is made to to verify the data was entered into the table 

query2 = " select artist, song, first_name, last_name from song_artist_user where user_id = 10 and session_id = 182"

try:
    rows = session.execute(query2)
except Exception as e:
    print(e)


printout = PrettyTable(["artist", "song", "first_name", "last_name"])
for row in rows:
    printout.add_row([row.artist, row.song, row.first_name, row.last_name])
print(printout)


+-------------------+------------------------------------------------------+------------+-----------+
|       artist      |                         song                         | first_name | last_name |
+-------------------+------------------------------------------------------+------------+-----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie   |    Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie   |    Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie   |    Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie   |    Cruz   |
+-------------------+------------------------------------------------------+------------+-----------+


In [14]:
# In order to get the first and last name of user that listened a specific song, the table created needs to have
# primary key in song id. However, this is not enough as different user would listen to the same song in different sessions
# so primary key on user id

query = """ create table if not exists user_song (song text, user_id int, first_name text, last_name text,  session_id int, 
            artist text,  gender text, item_in_session int, length float, level text, location text, 
            primary key ((song), user_id))"""
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [15]:
# Insert data into table for query 3

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = 'insert into %s (song, user_id, first_name, last_name, session_id, artist, gender,  \
                item_in_session, length, level, location)'%'user_song'
        query = query + " values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
        session.execute(query, (line[9], int(line[10]), line[1], line[4], int(line[8]), line[0], line[2], int(line[3]), float(line[5]), line[6], line[7]))

In [16]:
# SELECT statement is made to to verify the data was entered into the table

query3 = "select first_name, last_name from user_song where song = 'All Hands Against His Own'"

try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
printout = PrettyTable(["first_name", "last_name"])
for row in rows:
    printout.add_row([row.first_name, row.last_name])
print(printout)

+------------+-----------+
| first_name | last_name |
+------------+-----------+
|    Sara    |  Johnson  |
+------------+-----------+


### Drop the tables before closing out the sessions

In [17]:
# Drop the table before closing out the sessions
try:
    session.execute("drop table song_artist_session")
    session.execute("drop table song_artist_user")
    session.execute("drop table user_song")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()